In [1]:
!pip install ccxt


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import ccxt
import pandas as pd
import time
import sys
def fetch_all_data_ccxt(exchange_name, symbols, start_date='2010-01-01', end_date='2024-07-31', timeframe='1d'):
    try:
        exchange_class = getattr(ccxt, exchange_name)
        exchange = exchange_class()
    except AttributeError:
        print(f"Exchange {exchange_name} not found in ccxt library.")
        return None
    all_data = []
    total_rows = 0  # Initialize the total row count
    for symbol in symbols:
        try:
            since = exchange.parse8601(start_date)
            end = exchange.parse8601(end_date)
            ohlcv = []
            ohlcv_batch = exchange.fetch_ohlcv(symbol, timeframe, since=since, limit=5000)
            if len(ohlcv_batch) == 0:
                break
            ohlcv.extend(ohlcv_batch)
            rows_count = len(ohlcv) # count from the array storage
            total_rows = rows_count  # Increment total row count
            # total_rows += rows_count  # Increment total row count

            sys.stdout.write(f'\rProcessing {symbol} | Rows processed: {total_rows}')
            sys.stdout.flush()

            since = ohlcv_batch[-1][0] + exchange.parse_timeframe(timeframe)
            ohlcv.extend(ohlcv_batch)
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
            df['Timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.drop(columns='timestamp', inplace=True)
            df.set_index('Timestamp', inplace=True)
            df = df.sort_values(by='Timestamp')
            df['Tickers'] = symbol  # Add a column to indicate the symbol
            all_data.append(df)
        except Exception as e:
            print(f"\rError fetching data for {symbol}: {e}")

    combined_df = pd.concat(all_data)
    return combined_df
symbols = [ 'NMR/USDT',]
all_ccxt_data_huobi = fetch_all_data_ccxt(exchange_name='binance',symbols=symbols)
all_ccxt_data_huobi


Processing NMR/USDT | Rows processed: 1000

,Open,High,Low,Close,Volume,Tickers
Timestamp,,,,,,
2021-11-18,40.45,40.82,38.33,38.47,12852.54,NMR/USDT
2021-11-18,40.45,40.82,38.33,38.47,12852.54,NMR/USDT
2021-11-19,38.40,40.38,38.26,39.48,16094.89,NMR/USDT
2021-11-19,38.40,40.38,38.26,39.48,16094.89,NMR/USDT
2021-11-20,39.52,41.00,39.09,40.71,7123.52,NMR/USDT
...,...,...,...,...,...,...
2024-08-11,12.34,12.63,11.30,11.42,63121.23,NMR/USDT
2024-08-12,11.41,12.24,11.41,12.03,89265.09,NMR/USDT
2024-08-12,11.41,12.24,11.41,12.03,89265.09,NMR/USDT


In [1]:
import ccxt
import pandas as pd
import time

# exchange = ccxt.wazirx() # other AI tokens
# exchange = ccxt.bingx() # singularity : AGIX
# exchange = ccxt.exmo() # first tokens
exchange = ccxt.binance() # first tokens

symbols = [
    'AGIX/USDT',  # SingularityNET
    'OCEAN/USDT'
]
timeframe = '1d'  # 1-hour timeframe
limit = 3000  # Maximum limit per request

# Function to fetch OHLCV data in chunks
def fetch_ohlcv_in_chunks(symbol, timeframe, limit, num_entries):
    all_ohlcv = []
    since = None

    while len(all_ohlcv) < num_entries:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since, limit=limit)
        if not ohlcv:
            break
        all_ohlcv.extend(ohlcv)
        since = ohlcv[-1][0]  # Update 'since' to the last timestamp
        if len(ohlcv) < limit:
            break  # Exit if fewer entries are returned than the limit
        time.sleep(exchange.rateLimit / 1000)  # Respect rate limit

    return all_ohlcv[:num_entries]  # Return only the required number of entries

# Fetch data for each symbol
for symbol in symbols:
    ohlcv = fetch_ohlcv_in_chunks(symbol, timeframe, limit, 100000)

    # Extract the required columns
    timestamps = [candle[0] for candle in ohlcv]
    opens = [candle[1] for candle in ohlcv]
    highs = [candle[2] for candle in ohlcv]
    lows = [candle[3] for candle in ohlcv]
    closes = [candle[4] for candle in ohlcv]
    volumes = [candle[5] for candle in ohlcv]

    # Fetch the order book data
    order_book = exchange.fetch_order_book(symbol)
    bids = order_book['bids']
    asks = order_book['asks']

    # Fetch the trade data
    trades = exchange.fetch_trades(symbol)
    trade_prices = [trade['price'] for trade in trades]
    trade_volumes = [trade['amount'] for trade in trades]
    trade_timestamps = [trade['timestamp'] for trade in trades]

    # Fetch the market cap (if available)
    # market_cap = exchange.fetch_ticker(symbol)['marketCap']
    # market_cap = closes * circulating_supplies[symbol]
    # print(market_cap)

    # Calculate liquidity metrics (e.g., bid-ask spread)
    bid_price = order_book['bids'][0][0] if len(order_book['bids']) > 0 else None
    ask_price = order_book['asks'][0][0] if len(order_book['asks']) > 0 else None
    bid_ask_spread = (ask_price - bid_price) / bid_price if bid_price and ask_price else None

    data = pd.DataFrame({
        'Timestamp': timestamps,
        'Open': opens,
        'High': highs,
        'Low': lows,
        'Close': closes,
        'Volume': volumes,
        'Bid Price': [bid_price] * len(ohlcv),
        'Ask Price': [ask_price] * len(ohlcv),
        'Bid Depth': [bids[0][1] if bids else None] * len(ohlcv),
        'Ask Depth': [asks[0][1] if asks else None] * len(ohlcv),
        'Trade Prices': [trade_prices] * len(ohlcv),
        'Trade Volumes': [trade_volumes] * len(ohlcv),
        # 'Market Cap': [market_cap] * len(ohlcv),
        'Trade Timestamps': [trade_timestamps] * len(ohlcv),
        'Bid-Ask Spread': [bid_ask_spread] * len(ohlcv)
    })
    data.to_csv(f"ccxt/binance/numerai_symbols/{symbol.replace('/', '_')}.csv", index=False)

    print(f"Fetched {len(ohlcv)} entries for {symbol}")

Fetched 501 entries for AGIX/USDT
Fetched 1000 entries for OCEAN/USDT


In [2]:
coin_ids = {
    'BTC/USDT': 'bitcoin',
    'ETH/USDT': 'ethereum',
    'BNB/USDT': 'binancecoin',
    'SOL/USDT': 'solana',
    'XRP/USDT': 'ripple',
    'DOGE/USDT': 'dogecoin',
    'TON/USDT': 'toncoin',
    'ADA/USDT': 'cardano',
    'TRX/USDT': 'tron',
    'AVAX/USDT': 'avalanche-2',
    'SHIB/USDT': 'shiba-inu',
    'BCH/USDT': 'bitcoin-cash',
    'DOT/USDT': 'polkadot',
    'LINK/USDT': 'chainlink',
    'NEAR/USDT': 'near-protocol',
    'LEO/USDT': 'leo-token',
    'LTC/USDT': 'litecoin',
    'KAS/USDT': 'kaspa',
    'MATIC/USDT': 'matic-network',
    'PEPE/USDT': 'pepe',
    'UNI/USDT': 'uniswap',
    'ICP/USDT': 'internet-computer',
    'ETC/USDT': 'ethereum-classic',
    'APT/USDT': 'aptos',
    'XLM/USDT': 'stellar',
    'FET/USDT': 'fetch-ai',
    'XMR/USDT': 'monero',
    'MKR/USDT': 'maker',
    'STX/USDT': 'stacks',
    'FIL/USDT': 'filecoin',
    'OKB/USDT': 'okb',
    'MNT/USDT': 'montenegro',
    'CRO/USDT': 'crypto-com-chain',
    'ATOM/USDT': 'cosmos',
    'HBAR/USDT': 'hedera',
    'RENDER/USDT': 'render-token',
    'ARB/USDT': 'arbitrum',
    'TAO/USDT': 'tao',
    'IMX/USDT': 'immutable-x',
    'INJ/USDT': 'injective-protocol',
    'VET/USDT': 'vechain',
    'AR/USDT': 'arweave',
    'OP/USDT': 'optimism',
    'SUI/USDT': 'sui',
    'GRT/USDT': 'the-graph',
    'BONK/USDT': 'bonk',
    'AAVE/USDT': 'aave',
    'BGB/USDT': 'bgb',
    'FLOKI/USDT': 'floki-inu',
    'RUNE/USDT': 'thorchain',
    'JASMY/USDT': 'jasmy',
    'THETA/USDT': 'theta-network',
    'CHEEL/USDT': 'cheel',
    'FTM/USDT': 'fantom',
    'ALGO/USDT': 'algorand',
    'CORE/USDT': 'core',
    'FLOW/USDT': 'flow',
    'BSV/USDT': 'bitcoin-sv',
    'OM/USDT': 'om',
    'KCS/USDT': 'kucoin-shares',
    'BEAM/USDT': 'beam',
    'EOS/USDT': 'eos',
    'FLZ/USDT': 'flz',
    'BTT/USDT': 'bittorrent',
    'EGLD/USDT': 'elrond-erd-2',
    'AXS/USDT': 'axie-infinity',
    'ENS/USDT': 'ethereum-name-service',
    'QNT/USDT': 'quant-network',
    'HNT/USDT': 'harmony',
    'NEO/USDT': 'neo',
    'ORDI/USDT': 'ordi',
    'FLR/USDT': 'flare',
    'XEC/USDT': 'xec',
    'XTZ/USDT': 'tezos',
    'GALA/USDT': 'gala',
    'DYDX/USDT': 'dydx',
    'FTN/USDT': 'ftn',
    'CFX/USDT': 'conflux-network',
    'SAND/USDT': 'the-sandbox',
    'AKT/USDT': 'akt',
    'GT/USDT': 'gt',
    'MOG/USDT': 'mog',
    'NEXO/USDT': 'nexo',
    'WLD/USDT': 'wld',
    'AIOZ/USDT': 'aioz',
    'RON/USDT': 'ron',
    'XAUt/USDT': 'xaut',
    'RAY/USDT': 'raydium',
    'CHZ/USDT': 'chiliz',
    'MINA/USDT': 'mina-protocol',
    'ZEC/USDT': 'zcash',
    'ROSE/USDT': 'rose',
    'PENDLE/USDT': 'pendle',
    'SNX/USDT': 'synthetix',
    'BNX/USDT': 'bnx',
    'GNO/USDT': 'gnosis',
    'KLAY/USDT': 'klaytn',
    'DEXE/USDT': 'dexe',
    'IOTA/USDT': 'iota',
    'ASTR/USDT': 'astr',
    'CKB/USDT': 'ckb',
    'LPT/USDT': 'livepeer',
    'BTG/USDT': 'bitcoin-gold',
    'PAXG/USDT': 'paxos-standard',
    'APE/USDT': 'ape',
    'AXL/USDT': 'axl',
    'FTT/USDT': 'ftt',
    'SAFE/USDT': 'safe',
    'XDC/USDT': 'xdc-network',
    'COMP/USDT': 'compound',
    'KAVA/USDT': 'kava',
    'CAKE/USDT': 'pancakeswap',
    '1INCH/USDT': '1inch',
    'TFUEL/USDT': 'theta-fuel',
    'LUNC/USDT': 'terra-luna',
    'NFT/USDT': 'nft',
    'TWT/USDT': 'twt',
    'WEMIX/USDT': 'wemix',
    'IOTX/USDT': 'iotex',
    'MX/USDT': 'mx',
    'H2O/USDT': 'h2o',
    'SFP/USDT': 'sfp',
    'ANT/USDT': 'ant',
    'PEOPLE/USDT': 'people',
    'TURBO/USDT': 'turbo',
    'WOO/USDT': 'woo-network',
    'GAL/USDT': 'gal',
    'GMT/USDT': 'gmt',
    'BLUR/USDT': 'blur',
    'GLM/USDT': 'golem',
    'DASH/USDT': 'dash',
    'SUPER/USDT': 'super',
    'KSM/USDT': 'kusama',
    'CRV/USDT': 'curve-dao-token',
    'ZRX/USDT': '0x',
    'LUNA/USDT': 'luna',
    'ZIL/USDT': 'zilliqa',
    'PRIME/USDT': 'prime',
    'OSMO/USDT': 'osmosis',
    'SSV/USDT': 'ssv-network',
    'BDX/USDT': 'bdx',
    'ID/USDT': 'id',
    'SC/USDT': 'sc',
    'XRD/USDT': 'xrd',
    'ENJ/USDT': 'enjincoin',
    'BAT/USDT': 'basic-attention-token',
    'CVX/USDT': 'convex-finance',
    'HOT/USDT': 'holo',
    'ELF/USDT': 'elf',
    'JST/USDT': 'just',
    'TRAC/USDT': 'origintrail',
    'QTUM/USDT': 'qtum',
    'SKL/USDT': 'skale',
    'ARKM/USDT': 'arkm',
    'PAAL/USDT': 'paal',
    'GMX/USDT': 'gmx',
    'ETHW/USDT': 'ethw',
    'ILV/USDT': 'illuvium',
    'METIS/USDT': 'metis',
    'RVN/USDT': 'ravencoin',
    'PEPECOIN/USDT': 'pepecoin',
    'RSR/USDT': 'reserve-rights',
    'CFG/USDT': 'centrifuge',
    'GAS/USDT': 'gas',
    'BICO/USDT': 'biconomy',
    'T/USDT': 't',
    'POLYX/USDT': 'polyx',
    'FLUX/USDT': 'flux',
    'TRB/USDT': 'tellor',
    'UMA/USDT': 'uma',
    'XEM/USDT': 'nem',
    'DCR/USDT': 'decred',
    'CSPR/USDT': 'casper-network',
    'BAND/USDT': 'band-protocol',
    'XCH/USDT': 'chia',
    'LRC/USDT': 'loopring',
    'RLB/USDT': 'rlb',
    'ZIG/USDT': 'zigcoin',
    'YFI/USDT': 'yearn-finance',
    'ONE/USDT': 'harmony',
    'EDU/USDT': 'edu',
    'FXS/USDT': 'frax-share',
    'ABT/USDT': 'arcblock',
    'VANRY/USDT': 'vanry',
    'CHR/USDT': 'chromia',
    'SUSHI/USDT': 'sushi',
    'AMP/USDT': 'amp',
    'ACH/USDT': 'alchemix',
    'STORJ/USDT': 'storj',
    'BORG/USDT': 'borg',
    'CTC/USDT': 'ctc',
    'HONEY/USDT': 'honey',
    'BabyDoge/USDT': 'babydoge',
    'GLMR/USDT': 'glimmer',
    'YGG/USDT': 'yield-guild-games',
    'AUDIO/USDT': 'audius',
    'TEL/USDT': 'telcoin',
    'COTI/USDT': 'coti',
    'ALPH/USDT': 'alpha-finance',
    'API3/USDT': 'api3',
    'ICX/USDT': 'icon',
    'ZETA/USDT': 'zeta',
    'VENOM/USDT': 'venom',
    'SXP/USDT': 'swipe',
    'ZEN/USDT': 'zencash',
    'KDA/USDT': 'kadena',
    'RSS3/USDT': 'rss3',
    'NTRN/USDT': 'ntrn',
    'WAVES/USDT': 'waves',
    'LSK/USDT': 'lisk',
    'PCI/USDT': 'pci',
    'BAL/USDT': 'balancer',
    'WAXP/USDT': 'wax',
    'TRU/USDT': 'truefi',
    'ORCA/USDT': 'orca',
    'AGI/USDT': 'singularitynet',
    'PRO/USDT': 'pro',
    'BITCOIN/USDT': 'bitcoin',
    'AZERO/USDT': 'aleph-zero',
    'IOST/USDT': 'iost',
    'JOE/USDT': 'joe',
    'MAGIC/USDT': 'magic',
    'ONG/USDT': 'ontologynetwork',
    'BMX/USDT': 'bmx',
    'XNO/USDT': 'nervos-network',
    'XVS/USDT': 'venus',
    'AUCTION/USDT': 'auction',
    'POND/USDT': 'pond',
    'LISTA/USDT': 'lista',
    'LCX/USDT': 'lcx',
    'DGB/USDT': 'digibyte',
    'PROM/USDT': 'prom',
    'Gomining/USDT': 'gomining',
    'RLC/USDT': 'iexec-rlc',
    'C98/USDT': 'coin98',
    'CTSI/USDT': 'cartesi',
    'IQ/USDT': 'everipedia',
    'ZENT/USDT': 'zent',
    'KUJI/USDT': 'kujira',
    'POWR/USDT': 'power-ledger',
    'MOBILE/USDT': 'mobile',
    'PUNDIX/USDT': 'pundix',
    'GNS/USDT': 'gns',
    'SLP/USDT': 'smooth-love-potion',
    'HIVE/USDT': 'hive',
    'EVER/USDT': 'ever',
    'MVL/USDT': 'mvl',
    'BONE/USDT': 'bone',
    'RIF/USDT': 'rif',
    'BORA/USDT': 'bora',
    'DUSK/USDT': 'dusk-network',
    'HIGH/USDT': 'high',
    'ORAI/USDT': 'orai',
    'RDNT/USDT': 'radiant',
    'SFUND/USDT': 'seedify-fund',
    'LON/USDT': 'lon',
    'CVC/USDT': 'civic',
    'NMR/USDT': 'numeraire',
    'SUN/USDT': 'sun',
    'CELR/USDT': 'celer-network',
    'NOS/USDT': 'nos',
    'STMX/USDT': 'stmx',
    'STRAX/USDT': 'strax',
    'KNC/USDT': 'kyber-network',
    'WMT/USDT': 'wmt',
    'DAR/USDT': 'dar',
    'SNT/USDT': 'status',
    'MLK/USDT': 'mlk',
    'APEX/USDT': 'apex',
    'SPELL/USDT': 'spell',
    'ELON/USDT': 'dogelon-mars',
    'PHA/USDT': 'pha',
    'WILD/USDT': 'wild',
    'OAS/USDT': 'oas',
    'CTK/USDT': 'ctk',
    'WIN/USDT': 'win',
    'STEEM/USDT': 'steem',
    'MCOIN/USDT': 'mcoin',
    'MOVR/USDT': 'moonriver',
    'DODO/USDT': 'dodo',
    'HOOK/USDT': 'hook',
    'DENT/USDT': 'dent',
    'STPT/USDT': 'stpt',
    'MPLX/USDT': 'mplx',
    'ORBS/USDT': 'orbs',
    'NYM/USDT': 'nym',
    'LADYS/USDT': 'ladys',
    'VELO/USDT': 'velo',
    'PYR/USDT': 'vulcan-forged',
    'DAG/USDT': 'dag',
    'MPL/USDT': 'mpl',
    'MED/USDT': 'med',
    'SYN/USDT': 'synth',
    'HFT/USDT': 'hft',
    'CPOOL/USDT': 'cpool',
    'BOBO/USDT': 'bobo',
    'LQTY/USDT': 'liquity',
    'GODS/USDT': 'gods-unchained',
    'NAKA/USDT': 'naka',
    'ALEX/USDT': 'alex',
    'XPLA/USDT': 'xpla',
    'REQ/USDT': 'request-network',
    'GFI/USDT': 'gfi',
    'AURORA/USDT': 'aurora',
    'LOOM/USDT': 'loom-network',
    'SYS/USDT': 'syscoin',
    'ALICE/USDT': 'my-neighbor-alice',
    'PHB/USDT': 'phb',
    'MTL/USDT': 'metal',
    'SHDW/USDT': 'shadows',
    'LAT/USDT': 'lat',
    'OXT/USDT': 'oxen',
    'BLZ/USDT': 'bluzelle',
    'STG/USDT': 'stargate-finance',
    'CGPT/USDT': 'cgpt',
    'XYO/USDT': 'xyo',
    'STC/USDT': 'stc',
    'SCRT/USDT': 'secret',
    'EUL/USDT': 'euler',
    'AITECH/USDT': 'aitech',
    'MBX/USDT': 'mbx',
    'DESO/USDT': 'deso',
    'BAKE/USDT': 'bake',
    'FRONT/USDT': 'front',
    'AGLD/USDT': 'agld',
    'LMWR/USDT': 'lmwr',
    'LEVER/USDT': 'lever',
    'ARK/USDT': 'ark',
    'ARDR/USDT': 'ardor',
    'SNEK/USDT': 'snek',
    'TLOS/USDT': 'tlos',
    'ACA/USDT': 'aca',
    'BFIC/USDT': 'bfic',
    'BADGER/USDT': 'badger-dao',
    'HIFI/USDT': 'hifi-finance',
    'QKC/USDT': 'qkc',
    'UQC/USDT': 'uqc',
    'XVG/USDT': 'verge',
    'DAO/USDT': 'dao',
    'GHX/USDT': 'ghx',
    'BFC/USDT': 'bfc',
    'POKT/USDT': 'pokt',
    'HPO/USDT': 'hpo',
    'RAD/USDT': 'rad',
    'CREAM/USDT': 'cream-finance',
    'DIONE/USDT': 'dione',
    'RBN/USDT': 'ribbon-finance',
    'ARPA/USDT': 'arpa',
    'ACS/USDT': 'acs',
    'MBOX/USDT': 'mbox',
    'GTC/USDT': 'gtc',
    'ALI/USDT': 'ali',
    'RACA/USDT': 'raca',
    'ZCX/USDT': 'zcx',
    'OGN/USDT': 'origin-protocol',
    'ALPHA/USDT': 'alpha-finance',
    'RARE/USDT': 'rare',
    'NXRA/USDT': 'nxra',
    'MYRIA/USDT': 'myria',
    'GHST/USDT': 'ghst',
    'EWT/USDT': 'energy-web-token',
    'OGY/USDT': 'ogy',
    'FX/USDT': 'fx',
    'AIDOGE/USDT': 'aidoge',
    'TKO/USDT': 'tko',
    'ACX/USDT': 'acx',
    'WXT/USDT': 'wxt',
    'AOG/USDT': 'aog',
    'MAV/USDT': 'mav',
    'WRX/USDT': 'wrx',
    'LTO/USDT': 'lto-network',
    'MBL/USDT': 'mbl',
    'TLM/USDT': 'alien-worlds',
    'CBK/USDT': 'cbk',
    'FORTH/USDT': 'forth',
    'ERN/USDT': 'ern',
    'REI/USDT': 'rei',
    'BRISE/USDT': 'brise',
    'MLN/USDT': 'melon',
    'ETN/USDT': 'electroneum',
    'RARI/USDT': 'rari',
    'GEAR/USDT': 'gear',
    'META/USDT': 'meta',
    'CTXC/USDT': 'cortex',
    'DNX/USDT': 'dnx',
    'MAPO/USDT': 'mapo',
    'CLV/USDT': 'clv',
    'TET/USDT': 'tet',
    'LOOKS/USDT': 'looksrare',
    'XCN/USDT': 'xcn',
    'AERGO/USDT': 'aergo',
    'SWEAT/USDT': 'sweat',
    'BOSON/USDT': 'boson',
    'VR/USDT': 'vr',
    'BETA/USDT': 'beta',
    'SOLO/USDT': 'solo',
    'ROUTE/USDT': 'route',
    'DIMO/USDT': 'dimo',
    'ORN/USDT': 'orn',
    'DNT/USDT': 'district0x',
    'ALEPH/USDT': 'aleph',
    'BOBA/USDT': 'boba-network',
    'GOG/USDT': 'gog',
    'DMTR/USDT': 'dmtr',
    'DIA/USDT': 'dia',
    'COS/USDT': 'cos',
    'LOVELY/USDT': 'lovely-inu',
    'HUNT/USDT': 'hunt',
    'TT/USDT': 'tt',
    'DEP/USDT': 'dep',
    'FUN/USDT': 'funfair',
    'FLM/USDT': 'flamingo-finance',
    'RLY/USDT': 'rally',
    'AL/USDT': 'al',
    'DATA/USDT': 'data',
    'KMD/USDT': 'komodo',
    'VIC/USDT': 'vic',
    'ALCX/USDT': 'alchemix',
    'QUACK/USDT': 'quack',
    'DORA/USDT': 'dora',
    'GAFI/USDT': 'gafi',
    'STRK/USDT': 'strik',
    'LINA/USDT': 'linera',
    'UOS/USDT': 'ultra',
    'PDA/USDT': 'pda',
    'XCHNG/USDT': 'xchng',
    'STRX/USDT': 'strx',
    'COMBO/USDT': 'combo',
    'OMG/USDT': 'omg-network',
    'GFAL/USDT': 'gf',
    'CWIF/USDT': 'cwif',
    'VRTX/USDT': 'vrtx',
    'FIDA/USDT': 'fida',
    'WAN/USDT': 'wan',
    'ISIKC/USDT': 'isikc',
    'LEASH/USDT': 'leash',
    'SAUCE/USDT': 'sauce',
    'TRADE/USDT': 'trade',
    'COREUM/USDT': 'coreum',
    'UNFI/USDT': 'unfi',
    'VAI/USDT': 'vai',
    'PSG/USDT': 'paris-saint-germain-fan-token',
    'SAMO/USDT': 'samo',
    'NUM/USDT': 'num',
    'FCT/USDT': 'factom',
    'VRA/USDT': 'verasity',
    'AVA/USDT': 'ava',
    'ROG/USDT': 'rog',
    'NULS/USDT': 'nuls',
    'DEGO/USDT': 'dego',
    'LOKA/USDT': 'loka',
    'KEY/USDT': 'key',
    'FARM/USDT': 'farm',
    'REEF/USDT': 'reef',
    'CEEK/USDT': 'ceek',
    'NEON/USDT': 'neon',
    'MDT/USDT': 'mdt',
    'STT/USDT': 'stt',
    'GRS/USDT': 'groestlcoin',
    'BSW/USDT': 'biswap',
    'QUICK/USDT': 'quick',
    'NPT/USDT': 'npt',
    'LIME/USDT': 'lime',
    'ATLAS/USDT': 'atlas',
    'AMPL/USDT': 'ampleforth',
    'POLS/USDT': 'polkastarter',
    'ASD/USDT': 'asda',
    'GST/USDT': 'gala',
    'IRIS/USDT': 'iris',
    'KLIMA/USDT': 'klima',
    'MOC/USDT': 'moc',
    'VOXEL/USDT': 'voxel',
    'CERE/USDT': 'cere',
    'AMB/USDT': 'amb',
    'GMEE/USDT': 'gmee',
    'VGX/USDT': 'voyager-token',
    'BIFI/USDT': 'beefy-finance',
    'A8/USDT': 'a8',
    'KLV/USDT': 'klv',
    'GFT/USDT': 'gft',
    'TOMI/USDT': 'tomi',
    'AKRO/USDT': 'akropolis',
    'AQT/USDT': 'aqt',
    'ADX/USDT': 'adx',
    'DFI/USDT': 'dfi',
    'ZKL/USDT': 'zkl',
    'OPUL/USDT': 'opul',
    'HOPR/USDT': 'hopr',
    'MXC/USDT': 'mxc',
    'UTK/USDT': 'utk',
    'SABAI/USDT': 'sabai',
    'KP3R/USDT': 'kp3r',
    'BURGER/USDT': 'burger',
    'WING/USDT': 'wing',
    'BAR/USDT': 'bar',
    'CLORE/USDT': 'clore',
    'XCAD/USDT': 'xcad',
    'BOB/USDT': 'bob',
    'SRX/USDT': 'srx',
    'DF/USDT': 'df'
}
   

In [35]:
import ccxt
import os
import requests
import time
# Initialize the Binance exchange
exchange = ccxt.binance()
coin_ids={
    'AGIX/USDT': 'singularitynet',
    
}
# Function to fetch circulating supply from CoinGecko
def fetch_circulating_supply(coin_id):
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}'
    response = requests.get(url)
    data = response.json()
    print(coin_id , data)
    return data['market_data']['circulating_supply']

# Calculate market cap for each cryptocurrency
market_caps = []
path = 'ccxt/binance/numerai_symbols/'

for symbol, coin_id in coin_ids.items():
    for full_name in os.listdir(path):
        name = full_name.split(".")[0]
        # print(symbol.replace('/', '_') , name )
        if name == symbol.replace('/', '_'):
            try:
                # Fetch the current price from Binance
                ticker = exchange.fetch_ticker(symbol)
                current_price = ticker['last']
                
                # Fetch circulating supply from CoinGecko
                circulating_supply = fetch_circulating_supply(coin_id)
                print('cs:',circulating_supply ,'cp:',current_price )
                # Calculate market cap
                market_cap = current_price * circulating_supply
                
                # Append the results
                market_caps.append({
                    'Symbol': symbol,
                    'Current Price': current_price,
                    'Circulating Supply': circulating_supply,
                    'Market Cap': market_cap
                })
                # Open a file in write mode
                with open("output.txt", "a") as file:
                    # Write the text to the file
                    file.write('symbol: '+str(symbol) + '  |market cap:' )
                    file.write(str(market_cap) + '  |Current Price:' )
                    file.write(str(current_price) + '  |Circulating Supply:' )
                    file.write(str(circulating_supply) + '\n' )
            except Exception as e:
                print(f"Error fetching data for {symbol}: {e}")


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000024FD7C70290>>
Traceback (most recent call last):
  File "C:\Users\Malihe\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


#### numerai


In [3]:
!pip install numerapi
from numerapi import NumerAPI
napi = NumerAPI()


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# crypto
from numerapi import NumerAPI
napi = NumerAPI()

[f for f in napi.list_datasets() if f.startswith("crypto/v1.0")]

[
 'crypto/v1.0/live_universe.parquet',
 'crypto/v1.0/train_targets.parquet',
]

# Download the training data
napi.download_dataset("crypto/v1.0/train_targets.parquet")
napi.download_dataset("crypto/v1.0/live_universe.parquet")

In [156]:
import pandas as pd
train_targets = pd.read_parquet("crypto/v1.0/train_targets.parquet")
live_universe = pd.read_parquet("crypto/v1.0/live_universe.parquet")


In [157]:
live_universe.to_csv('live_universe.csv', index=False)
train_targets.to_csv('train_targets.csv', index=False)

In [158]:
train_targets

,symbol,date,target
0,0xBTC,2020-06-01,0.00
1,ABBC,2020-06-01,0.50
2,ABT,2020-06-01,0.50
3,ADA,2020-06-01,0.75
4,AE,2020-06-01,0.50
...,...,...,...
496,ZENT,2024-06-26,0.25
497,ZETA,2024-06-26,0.25
498,ZIG,2024-06-26,1.00
499,ZIL,2024-06-26,0.50


In [159]:
live_universe

,symbol
0,BTC
11,ETH
49,BNB
196,SOL
2,XRP
...,...
1046,CLORE
802,XCAD
1031,BOB
832,SRX


In [160]:
train_targets['symbol'].nunique()

1433

In [161]:
live_universe['symbol'].nunique()


500

In [32]:
live_universe['id'] = live_universe['symbol']+'/USDT'
live_universe['id']

0         BTC/USDT
11        ETH/USDT
49        BNB/USDT
196       SOL/USDT
2         XRP/USDT
           ...    
1046    CLORE/USDT
802      XCAD/USDT
1031      BOB/USDT
832       SRX/USDT
649        DF/USDT
Name: id, Length: 500, dtype: object

#### yfinance

In [1]:
! pip install yfinance


[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import yfinance as yf

btc_data = yf.download('BTC-USD', start='2020-01-01', end='2024-01-01')
btc_data['Returns'] = btc_data['Adj Close'].pct_change()


[*********************100%%**********************]  1 of 1 completed


In [ ]:

# Fetch the 3-month Treasury bill rate (risk-free rate)
t_bill = yf.download('^IRX', start='2020-01-01', end='2024-01-01')
t_bill['Risk-Free Rate'] = t_bill['Adj Close'] / 100  # Convert to a decimal

# Display the risk-free rate
print(t_bill['Risk-Free Rate'])

### google trends

In [5]:
!pip install pytrends

  Using cached pytrends-4.9.2-py3-none-any.whl (15 kB)



[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
pytrends = TrendReq()
pytrend.build_payload(kw_list=['bitcoin'] ,timeframe='today 5-y')
df = pytrend.interest_over_time()
# df = pytrend.interest_by_region()
df = df.reset_index()
df.head(10)


,date,bitcoin,isPartial
0,2019-08-04,20,False
1,2019-08-11,17,False
2,2019-08-18,16,False
3,2019-08-25,14,False
4,2019-09-01,15,False
5,2019-09-08,13,False
6,2019-09-15,12,False
7,2019-09-22,17,False
8,2019-09-29,13,False
9,2019-10-06,13,False


In [34]:
from pytrends.request import TrendReq
import pandas as pd
import time
import os
import warnings
warnings.filterwarnings("ignore")
pytrend = TrendReq()
all_interest_data = pd.DataFrame()
# os.mkdir('google trends/')
for coin, keyword in coin_ids.items():
# for keyword in ['ocean protocol', 'singularitynet', 'numerai', ]:
    try:
        pytrend.build_payload(kw_list=[keyword], timeframe='today 5-y')
        
        df = pytrend.interest_over_time()
        df = df.reset_index()

        df.to_csv(f'google trends/{keyword}.csv', index=False)

        print(f"Successfully fetched data for {keyword}")

    except Exception as e:
        print(f"An error occurred for {keyword}: {e}")
        time.sleep(1)  # Optional: Add a delay to avoid hitting rate limits




Successfully fetched data for ocean protocol
Successfully fetched data for singularitynet
Successfully fetched data for numerai


barear token
AAAAAAAAAAAAAAAAAAAAAHeCvQEAAAAAAycv8kXSDazYR%2Bq77ODp1%2Bm2cPA%3D9gRGIyZGOpgc87Oi41pyb6ZsE8dEsnkdCkHFh0gx2GfkGeAneo

access token 
2602263745-1tAIXKVS5fh1Iyfa7BxHanCzcgBJcUrv121RT5F
access token Secret
4O4jxI4d1yl5eboRspLZsUGFWDu0vCOts2LKuI44ZQ5D8

#### sentiment

In [1]:
! pip install tweepy

     -------------------------------------- 98.5/98.5 kB 209.2 kB/s eta 0:00:00
     ------------------------------------ 151.7/151.7 kB 565.8 kB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tweepy
client = tweepy.Client("AAAAAAAAAAAAAAAAAAAAAHeCvQEAAAAAAycv8kXSDazYR%2Bq77ODp1%2Bm2cPA%3D9gRGIyZGOpgc87Oi41pyb6ZsE8dEsnkdCkHFh0gx2GfkGeAneo")

In [3]:
import tweepy
import pandas as pd

# Define your search parameters
query = "ocean protocol"  # Replace with your search term
start_time = "2020-01-01T00:00:00Z"  # Start time in ISO 8601 format
end_time = "2024-01-08T00:00:00Z"    # End time in ISO 8601 format

# Use the Paginator to handle pagination
tweets = tweepy.Paginator(
    client.search_all_tweets,
    query=query,
    start_time=start_time,
    end_time=end_time,
    max_results=100  # Adjust the number of results per request as needed
).flatten(limit=500)  # Adjust the limit for total tweets to retrieve

# Create a DataFrame to store the tweet data
tweets_data = []

for tweet in tweets:
    tweets_data.append({
        'username': tweet.author_id,
        'created_at': tweet.created_at,
        'text': tweet.text,
        'tweet_id': tweet.id
    })

# Convert to DataFrame for easier analysis
tweets_df = pd.DataFrame(tweets_data)

# Display the DataFrame
print(tweets_df)

Forbidden: 403 Forbidden
When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.

vaderSentiment

In [1]:
! pip install vaderSentiment

     ------------------------------------ 126.0/126.0 kB 322.3 kB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sentences = ["hate it"," i love this "]
analyzer = SentimentIntensityAnalyzer()
for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<65} {}".format(sentence, str(vs)))

hate it---------------------------------------------------------- {'neg': 0.787, 'neu': 0.213, 'pos': 0.0, 'compound': -0.5719}
 i love this ---------------------------------------------------- {'neg': 0.0, 'neu': 0.323, 'pos': 0.677, 'compound': 0.6369}
